This notebook contains an analysis workflow for refining textures in diffraction spectra using MAUD (http://maud.radiographema.eu/)

## Import packages

In [7]:
from decimal import Decimal
import pathlib

from tqdm.notebook import tqdm
import numpy as np

## Preparing the DAWN spectra

In DAWN the data must be caked with the x-axis chosen as pixel number (pixels) and saved as a .dat file.

It is not possible to adjust data using x-axis as the two-theta angle (degrees).

**Notes on caking in DAWN**

- Import tiffs using either .nxs file (for standard 2 Hz acquisition) or as .dat file (for faster acquisitions) into ‘data slice viewer’ within the ‘processing’ tab.

- Select pilatus, image 2D, finish.

- For **cake remapping** (used in MAUD for texture recalculation and for single peak profile analysis in python), in ‘processing’ window;
    - Import detector calibration (calibration_output.nxs)
    - Threshold mask, set lower to 0.0
    - Cake remapping, azimuthal range 2.5, -357.5, number of azimuthal bins 72, number of bins blank, pixel splitting tick, radial range blank, X axis pixel (MAUD) or angle (SPP)
    - Export to text file, file extension .dat, choose output directory, pad with zeros 5

- For **azimuthal integration** (used in TOPAS for volume fraction), in ‘processing’ window;
    - Import detector calibration (calibration_output.nxs)
    - Threshold mask, set lower to 0.0
    - Azimuthal integration, azimuthal range blank, number of bins blank, pixel splitting tick, radial range blank, X axis angle 
    - Export to text file, file extension .xy, choose output directory, pad with zeros 5
        
*Data can also be caked using pyFAI (https://pyfai.readthedocs.io/en/latest/#)*

## Instructions for setting up MAUD batch mode

Before beginning the batch mode analysis a **template .par file** must be set up in MAUD.

To do this, create an **'instrument type'** in MAUD;

- Create a .esg file containing the calibratrion data using the script below.
- Load the .esg calibration file into MAUD.
- Follow online videos to define the instrument (https://www.youtube.com/user/MaudRietveldProgram/videos)

*Note, the .dat files have undergone claibration in DAWN, so ImageJ isn't needed to load the tiffs, which would lead to a different calibration in MAUD.*

Then, create a **template .par file** in MAUD;

- Create a .esg file containing experimental data using the scipt.
- Load this into MAUD.
- Select the new instrument type.
- Setup the refinement;
    - Set data range ~ 2.2 to 9.9
    - Add extra background parameter.
    - Add alpha and beta cifs.
    - Set Biso to 0.5
    - Set approximate volume fraction.
    - Manually adjust crystal paramters to fit peak positions.
    - Auto refine background, free scale factors and cell parameters. *Note, leave beam centre fixed.*
    - Auto refine volume fraction.
    - Auto refine crystal size and micro-strain.
    - Add E-WIMV texture for alpha-phase (set ODF resolution to 5 or 10 degree resolution).
    - Fix crystal size and micro-strain and turn on texture refinement (but, don't refine).
    - *Note, to refine beta-phase, untick ODF refineable on alpha-phase.*
    
- Save this as a template .par file i.e. 'start_065.par'

## User parameters for calibration file

In [2]:
# user inputs (in mm)
detector_distance = 1200
pixel_size = 0.296

# the formatting of the name of the input files
file_prefix = 'DAWN_1200mm_calibration'
dir_name = 'calibration/'

# this specifies the angle of the first cake in the data file.
start_angle = 0

# where the .esg file is written
output_folder = pathlib.Path("calibration")

## Write out a single .esg calibration file for MAUD

In [3]:
# caked synchrotron data from DAWN
input_file = dir_name + file_prefix + '.dat'
input_data = np.loadtxt(input_file)

# read the pixel spacing from the data file and multiply by pixel size to get pixel positions in mm
pixel_list = input_data[:, 0] * pixel_size

cake_spectrum = input_data[:, 1:]
number_of_cakes = cake_spectrum.shape[1]
cake_width = 360 / number_of_cakes

# name and open the output MAUD data file
output_folder.mkdir(exist_ok=True)
output_path = output_folder / pathlib.Path(file_prefix + '.esg')

with open(output_path, 'w') as output_file:

    # write metadata for the top of the file
    output_file.write('_pd_block_id noTitle|#0\n'
    '_diffrn_detector Image Plate\n'
    '_diffrn_detector_type Image Plate\n'
    '_pd_meas_step_count_time ?\n'
    '_diffrn_measurement_method ?\n'
    '_diffrn_measurement_distance_unit mm\n'
    '_pd_instr_dist_spec/detc {:.4f}\n'.format(detector_distance))
    output_file.write('_diffrn_radiation_wavelength ?\n'
    '_diffrn_source_target ?\n'
    '_diffrn_source_power ?\n'
    '_diffrn_source_current ?\n'
    '_pd_meas_angle_omega 0.0\n'
    '_pd_meas_angle_chi 0.0\n'
    '_pd_meas_angle_phi 0.0\n'
    '_riet_par_spec_displac_x 0\n'
    '_riet_par_spec_displac_y 0\n'
    '_riet_par_spec_displac_z 0\n'
    '_riet_meas_datafile_calibrated false\n'
    '_pd_meas_angle_eta {:.1f}\n\n'.format(start_angle))
    output_file.write('loop_\n'
    '_pd_proc_2theta_corrected\n'
    '_pd_meas_intensity_total\n')

    # write the first cake intensity data
    for i in range(len(pixel_list)-1,-1,-1):
        output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[i][0]))

    # write all the other cake data with additional info
    for cake_number in range(1, number_of_cakes, 1):    
        output_file.write('\n_pd_block_id noTitle|#{:.0f}\n\n'.format(cake_number))
        cake_angle = start_angle + cake_number * cake_width
        output_file.write('_pd_meas_angle_eta {:.1f}\n\n'.format(cake_angle))
        output_file.write('loop_\n'
        '_pd_proc_2theta_corrected\n'
        '_pd_meas_intensity_total\n')

        # write cake intensity data for each cake
        for i in range(len(pixel_list)-1, -1, -1):
            output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[i][cake_number]))
                
print(f"Calibration .esg data file written to '{output_folder}' folder.")

Calibration .esg data file written to 'calibration' folder.


## User parameters for preparing analysis files

In [4]:
# user inputs (in mm)
detector_distance = 1194.8865 # use the calibrated detector distance
pixel_size = 0.296

# test number is put in the name of the input file
test_number = 65

# the formatting of the name of the input files
file_prefix = 'adc_{:03d}_TI64_NDload_900C_15mms'.format(test_number)
dir_name = 'data/' + file_prefix + '_ascii/'

# this specifies the angle of the first cake in the data file.
start_angle = 0

# number and spacing of files to read
start = 0
end = 10
step = 1

# where the .esg files are written
output_folder = pathlib.Path("analysis")

## Loop through multiple input files to output .esg data files for MAUD

Normally, we have many analysis files spaced in time. We can read these in a loop and produce one .esg data file for each input file.

In this case the output files are written into a folder called 'analysis' and saved in the format MAUD_065_00001.esg where 065 is experiment number and 00001 is test number.

In [5]:
# loop through the data using the image number of the files
for image_number in tqdm(range(start, end + 1, step)):
    
    # caked synchrotron data from DAWN
    input_file = dir_name + file_prefix + '_{:05d}.dat'.format(image_number)
    input_data = np.loadtxt(input_file)
    # Read the pixel spacing from the data file and multiply by pixel size to get pixel positions in mm
    pixel_list = input_data[:, 0] * pixel_size
    
    cake_spectrum = input_data[:, 1:]
    number_of_cakes = cake_spectrum.shape[1]
    cake_width = 360 / number_of_cakes
    
    # name and open the output MAUD data file
    output_folder.mkdir(exist_ok=True)
    output_path = output_folder / pathlib.Path('MAUD_{:03d}_{:05d}.esg'.format(test_number,image_number))
    
    with open(output_path, 'w') as output_file:

        # write metadata for the top of the file
        output_file.write('_pd_block_id noTitle|#0\n'
        '_diffrn_detector Image Plate\n'
        '_diffrn_detector_type Image Plate\n'
        '_pd_meas_step_count_time ?\n'
        '_diffrn_measurement_method ?\n'
        '_diffrn_measurement_distance_unit mm\n'
        '_pd_instr_dist_spec/detc {:.4f}\n'.format(detector_distance))
        output_file.write('_diffrn_radiation_wavelength ?\n'
        '_diffrn_source_target ?\n'
        '_diffrn_source_power ?\n'
        '_diffrn_source_current ?\n'
        '_pd_meas_angle_omega 0.0\n'
        '_pd_meas_angle_chi 0.0\n'
        '_pd_meas_angle_phi 0.0\n'
        '_riet_par_spec_displac_x 0\n'
        '_riet_par_spec_displac_y 0\n'
        '_riet_par_spec_displac_z 0\n'
        '_riet_meas_datafile_calibrated false\n'
        '_pd_meas_angle_eta {:.1f}\n\n'.format(start_angle))
        output_file.write('loop_\n'
        '_pd_proc_2theta_corrected\n'
        '_pd_meas_intensity_total\n')

        # write the first cake intensity data
        for i in range(len(pixel_list)-1,-1,-1):
            output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[i][0]))

        # write all the other cake data with additional info
        for cake_number in range(1, number_of_cakes, 1):    
            output_file.write('\n_pd_block_id noTitle|#{:.0f}\n\n'.format(cake_number))
            cake_angle = start_angle + cake_number * cake_width
            output_file.write('_pd_meas_angle_eta {:.1f}\n\n'.format(cake_angle))
            output_file.write('loop_\n'
            '_pd_proc_2theta_corrected\n'
            '_pd_meas_intensity_total\n')

            # write cake intensity data for each cake
            for i in range(len(pixel_list)-1, -1, -1):
                output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[i][cake_number]))
                
print(f"{int((end + 1 - start) / step)} .esg data files written to '{output_folder}' folder.")


11 .esg data files written to 'analysis' folder.


## Output many .par analysis files for MAUD batch mode analysis

Write out a number of .par analysis files to be refined using MAUD batch mode. 


Use a starting MAUD .par analysis file to swap the data into. This file will have been manually refined in MAUD and setup ready to refine in batch mode. It is assumed that this start MAUD par file is named in the format 'start_065.par', where 065 is the experiment number.


The output files are written into an 'analysis' folder and saved in the format MAUD_065_00001.par where 065 is experiment number and 00001 is test number. This analysis folder will then be used in the subsequent batch mode analysis.

In [ ]:
# user inputs (in mm)
pixel_size = 0.296

# test number is put in the name of the input file
test_number = 65

# the formatting of the name of the input files
file_prefix = 'adc_{:03d}_TI64_NDload_900C_15mms'.format(test_number)
dir_name = 'data/' + file_prefix + '_ascii/'

# number and spacing of files to read
start = 0
end = 10
step = 1

# where the .esg files are written
output_folder = pathlib.Path("analysis")

# path to the start_***.par file
par_dir_name = 'data/par/'

# name of the data (.esg files) to be replaced in the MAUD start_***.par file
previous_esg_filename = 'MAUD_{:03d}_00000'.format(test_number)

# MAUD file to swap the data into
template_par = par_dir_name + 'start_{:03d}.par'.format(test_number)

In [11]:
for image_number in tqdm(range(start, end + 1, step)):
    
    # caked synchrotron data from DAWN
    input_file = dir_name + file_prefix + '_{:05d}.dat'.format(image_number)
    input_data = np.loadtxt(input_file)
    # read the pixel spacing from the data file and multiply by pixel size to get pixel positions in mm
    pixel_list = input_data[:, 0] * pixel_size
    
    cake_spectrum = input_data[:, 1:]
    number_of_cakes = cake_spectrum.shape[1]
    cake_width = 360 / number_of_cakes
   
    # output MAUD analysis file
    output_par_path = output_folder / pathlib.Path(f'MAUD_{test_number:03d}_{image_number:05d}.par')
    
    with open(template_par, 'r') as template_par_file, open(output_par_path, 'w') as new_par_file:
        cake_number = 0
        line = template_par_file.readline() 
            
        while line:
            # rename the datafiles in the .par file
            if previous_esg_filename in line:
                new_esg_filename = 'MAUD_{:03d}_{:05d}'.format(test_number, image_number)
                new_line = line.replace(previous_esg_filename, new_esg_filename)
                new_par_file.write(new_line)
                
            # replace the data when this line is found
            elif '_pd_meas_position _pd_meas_intensity_total _pd_meas_intensity_sigma' in line:
                new_par_file.write(line)
                
                # write the new data in place
                for j in range(len(pixel_list)-1, -1, -1):
                    template_par_file.readline()
                    new_par_file.write('{:.3f}\t{:.8f}\t1.0\n'.format(pixel_list[j], cake_spectrum[j][cake_number]))
                cake_number += 1
            
            # write the rest of the file
            else:
                new_par_file.write(line)
            
            line = template_par_file.readline()
            
print(f"Written {int((end + 1 - start) / step)} files to '{output_folder}' folder.")

data_global

_audit_creation_date 'Thu Oct 10 12:48:47 BST 2019'

_audit_creation_method 'Maud, version 2.92'

_audit_update_record 'Thu Oct 10 12:58:52 BST 2019'

_computing_structure_refinement 'Maud, version 2.92'

_refine_ls_R_factor_all 0.35874194

_refine_ls_wR_factor_all 0.4355284

_refine_ls_goodness_of_fit_all 0.33484024

_publ_contact_author_name 'Luca Lutterotti'

_publ_section_title 'Put a title here'

_pd_proc_ls_extract_int 'end of iteration'

_pd_proc_ls_texture_comp 'end of iteration'

_computing_reduce_memory_occ true

_pd_proc_ls_theoretical_weight false

_pd_proc_ls_extract_pos 'end of iteration'

_pd_proc_ls_strain_comp 'end of iteration'

_pd_proc_ls_extract_Fhkl 'end of iteration'

_pd_proc_ls_Fhkl_comp 'end of iteration'

_pd_proc_ls_weight_scheme sqrt

_refine_ls_weighting_scheme WgtSS

_refine_ls_WSS_factor 29153.68

_maud_store_spectra_with_analysis true

_riet_remove_phases_under 0.001

_riet_refine_cell_over 0.1

_riet_refine_sizestrain_over 0.1

_riet_refin


#end_custom_object_intensity_data





#custom_object_texture_factors

_rita_texture_points_number 1_rita_texture_radiations_number 1



data_phase_alpha



loop_

_refln_index_h _refln_index_k _refln_index_l _rita_texture_factor_meas _rita_texture_factor_calc _rita_texture_point_index _rita_texture_radiation_index

1 0 0 1.0 1.0 0 0

0 0 2 1.0 1.0 0 0

1 0 1 1.0 1.0 0 0

1 0 2 1.0 1.0 0 0

1 1 0 1.0 1.0 0 0

1 0 3 1.0 1.0 0 0

2 0 0 1.0 1.0 0 0

1 1 2 1.0 1.0 0 0

2 0 1 1.0 1.0 0 0

0 0 4 1.0 1.0 0 0

2 0 2 1.0 1.0 0 0

1 0 4 1.0 1.0 0 0

2 0 3 1.0 1.0 0 0

2 1 0 1.0 1.0 0 0

2 1 1 1.0 1.0 0 0

1 1 4 1.0 1.0 0 0

2 1 2 1.0 1.0 0 0

1 0 5 1.0 1.0 0 0

2 0 4 1.0 1.0 0 0

3 0 0 1.0 1.0 0 0

3 0 1 1.0 1.0 0 0

2 1 3 1.0 1.0 0 0





data_phase_beta



loop_

_refln_index_h _refln_index_k _refln_index_l _rita_texture_factor_meas _rita_texture_factor_calc _rita_texture_point_index _rita_texture_radiation_index

1 1 0 1.0 1.0 0 0

2 0 0 1.0 1.0 0 0

2 1 1 1.0 1.0 0 0

2 2 0 1.0 1.0 0 0

3 1

_riet_par_spec_displac_x_2R 0 #min -10.0 #max 10.0

_riet_par_spec_displac_z_2R 0 #min -10.0 #max 10.0

_riet_par_fluo_diffr_scale 1.0 #positive #min 0.0 #max 100.0

_rita_shape_abs_velocity_corr_factor 0 #positive #min 0.0 #max 10000.0





#custom_object_intensity_data

_pd_meas_number_of_points 1027

_riet_meas_datafile_calibrated false

_riet_meas_datafile_dspacing_based false

_riet_meas_datafile_energy_dispersive false

_riet_meas_datafile_constant_step false

loop_

_pd_meas_position _pd_meas_intensity_total _pd_meas_intensity_sigma



#end_custom_object_intensity_data





#custom_object_texture_factors

_rita_texture_points_number 1_rita_texture_radiations_number 1



data_phase_alpha



loop_

_refln_index_h _refln_index_k _refln_index_l _rita_texture_factor_meas _rita_texture_factor_calc _rita_texture_point_index _rita_texture_radiation_index

1 0 0 1.0 1.0 0 0

0 0 2 1.0 1.0 0 0

1 0 1 1.0 1.0 0 0

1 0 2 1.0 1.0 0 0

1 1 0 1.0 1.0 0 0

1 0 3 1.0 1.0 0 0

2 0 0 1.0 1.0 0 0






loop_

_refln_index_h _refln_index_k _refln_index_l _rita_texture_factor_meas _rita_texture_factor_calc _rita_texture_point_index _rita_texture_radiation_index

1 1 0 1.0 1.0 0 0

2 0 0 1.0 1.0 0 0

2 1 1 1.0 1.0 0 0

2 2 0 1.0 1.0 0 0

3 1 0 1.0 1.0 0 0

2 2 2 1.0 1.0 0 0

3 2 1 1.0 1.0 0 0





#end_custom_object_texture_factors





#end_subordinateObject_MAUD_065_00000.esg(30)





#subordinateObject_MAUD_065_00000.esg(31)



_riet_meas_datafile_name 'MAUD_065_00000.esg(31)'



_riet_meas_datafile_format ?

_pd_meas_angle_omega 0.0

_pd_meas_angle_chi 0.0

_pd_meas_angle_phi 0.0

_pd_meas_angle_eta 310.0

_pd_meas_angle_2theta 0.0

_pd_meas_energy_kev 0.0

_riet_meas_datafile_compute true

_riet_meas_datafile_fitting false

_pd_meas_detector_id none

_pd_meas_step_count_time 1.0

_pd_meas_units_of_intensity counts

_riet_meas_datafile_as_background false

_riet_meas_data_group_count 1

_riet_datafile_type 0

_riet_datafile_save_custom ?

_pd_meas_image_id -1

_riet_background_in



data_global

_audit_creation_date 'Thu Oct 10 12:48:47 BST 2019'

_audit_creation_method 'Maud, version 2.92'

_audit_update_record 'Thu Oct 10 12:58:52 BST 2019'

_computing_structure_refinement 'Maud, version 2.92'

_refine_ls_R_factor_all 0.35874194

_refine_ls_wR_factor_all 0.4355284

_refine_ls_goodness_of_fit_all 0.33484024

_publ_contact_author_name 'Luca Lutterotti'

_publ_section_title 'Put a title here'

_pd_proc_ls_extract_int 'end of iteration'

_pd_proc_ls_texture_comp 'end of iteration'

_computing_reduce_memory_occ true

_pd_proc_ls_theoretical_weight false

_pd_proc_ls_extract_pos 'end of iteration'

_pd_proc_ls_strain_comp 'end of iteration'

_pd_proc_ls_extract_Fhkl 'end of iteration'

_pd_proc_ls_Fhkl_comp 'end of iteration'

_pd_proc_ls_weight_scheme sqrt

_refine_ls_weighting_scheme WgtSS

_refine_ls_WSS_factor 29153.68

_maud_store_spectra_with_analysis true

_riet_remove_phases_under 0.001

_riet_refine_cell_over 0.1

_riet_refine_sizestrain_over 0.1

_riet_ref

...

_pd_meas_angle_phi 0.0

_pd_meas_angle_eta 220.0

_pd_meas_angle_2theta 0.0

_pd_meas_energy_kev 0.0

_riet_meas_datafile_compute true

_riet_meas_datafile_fitting false

_pd_meas_detector_id none

_pd_meas_step_count_time 1.0

_pd_meas_units_of_intensity counts

_riet_meas_datafile_as_background false

_riet_meas_data_group_count 1

_riet_datafile_type 0

_riet_datafile_save_custom ?

_pd_meas_image_id -1

_riet_background_interpolated_manual false

_pd_meas_datetime_initiated ?

_pd_proc_ls_datafile_weight 1.0

_riet_use_count_time false

_riet_chebyshev_polynomial_background ?

_pd_meas_counts_monitor 1.0 #positive #min 0.0 #max 10.0

_riet_par_spec_displac_x_2R 0 #min -10.0 #max 10.0

_riet_par_spec_displac_z_2R 0 #min -10.0 #max 10.0

_riet_par_fluo_diffr_scale 1.0 #positive #min 0.0 #max 100.0

_rita_shape_abs_velocity_corr_factor 0 #positive #min 0.0 #max 10000.0





#custom_object_intensity_data

_pd_meas_number_of_points 1027

_riet_meas_datafile_calibrated false

_riet_meas_






#end_custom_object_odf





#end_subordinateObject_E-WIMV





#subordinateObject_Delft



_riet_size_strain_model 'Delft'





#end_subordinateObject_Delft





#subordinateObject_Isotropic



_riet_sizestrain_sym_model 'Isotropic'



_riet_par_cryst_size 1000.98517 #positive #min 50.0 #max 5000.0

_riet_par_rs_microstrain 1.7671756E-7 #positive #min 0.0 #max 0.005



#end_subordinateObject_Isotropic





#subordinateObject_none abm



_riet_antiphase_boundary 'none abm'





#end_subordinateObject_none abm





#subordinateObject_none pd



_riet_planar_defect 'none pd'





#end_subordinateObject_none pd





#subordinateObject_no magnetic



_riet_magnetic_structure_model 'no magnetic'





#end_subordinateObject_no magnetic





#subordinateObject_no strain



_riet_par_strain_model 'no strain'





#end_subordinateObject_no strain





#subordinateObject_No microabsorption



_riet_micro_absorption_model 'No microabsorption'





#end_subordinateObject_No microabsorption






1 0 2 1.0 1.0 0 0

1 1 0 1.0 1.0 0 0

1 0 3 1.0 1.0 0 0

2 0 0 1.0 1.0 0 0

1 1 2 1.0 1.0 0 0

2 0 1 1.0 1.0 0 0

0 0 4 1.0 1.0 0 0

2 0 2 1.0 1.0 0 0

1 0 4 1.0 1.0 0 0

2 0 3 1.0 1.0 0 0

2 1 0 1.0 1.0 0 0

2 1 1 1.0 1.0 0 0

1 1 4 1.0 1.0 0 0

2 1 2 1.0 1.0 0 0

1 0 5 1.0 1.0 0 0

2 0 4 1.0 1.0 0 0

3 0 0 1.0 1.0 0 0

3 0 1 1.0 1.0 0 0

2 1 3 1.0 1.0 0 0





data_phase_beta



loop_

_refln_index_h _refln_index_k _refln_index_l _rita_texture_factor_meas _rita_texture_factor_calc _rita_texture_point_index _rita_texture_radiation_index

1 1 0 1.0 1.0 0 0

2 0 0 1.0 1.0 0 0

2 1 1 1.0 1.0 0 0

2 2 0 1.0 1.0 0 0

3 1 0 1.0 1.0 0 0

2 2 2 1.0 1.0 0 0

3 2 1 1.0 1.0 0 0





#end_custom_object_texture_factors





#end_subordinateObject_MAUD_065_00000.esg(3)





#subordinateObject_MAUD_065_00000.esg(4)



_riet_meas_datafile_name 'MAUD_065_00000.esg(4)'



_riet_meas_datafile_format ?

_pd_meas_angle_omega 0.0

_pd_meas_angle_chi 0.0

_pd_meas_angle_phi 0.0

_pd_meas_angle_eta 40.0

_p

_pd_meas_number_of_points 1027

_riet_meas_datafile_calibrated false

_riet_meas_datafile_dspacing_based false

_riet_meas_datafile_energy_dispersive false

_riet_meas_datafile_constant_step false

loop_

_pd_meas_position _pd_meas_intensity_total _pd_meas_intensity_sigma



#end_custom_object_intensity_data





#custom_object_texture_factors

_rita_texture_points_number 1_rita_texture_radiations_number 1



data_phase_alpha



loop_

_refln_index_h _refln_index_k _refln_index_l _rita_texture_factor_meas _rita_texture_factor_calc _rita_texture_point_index _rita_texture_radiation_index

1 0 0 1.0 1.0 0 0

0 0 2 1.0 1.0 0 0

1 0 1 1.0 1.0 0 0

1 0 2 1.0 1.0 0 0

1 1 0 1.0 1.0 0 0

1 0 3 1.0 1.0 0 0

2 0 0 1.0 1.0 0 0

1 1 2 1.0 1.0 0 0

2 0 1 1.0 1.0 0 0

0 0 4 1.0 1.0 0 0

2 0 2 1.0 1.0 0 0

1 0 4 1.0 1.0 0 0

2 0 3 1.0 1.0 0 0

2 1 0 1.0 1.0 0 0

2 1 1 1.0 1.0 0 0

1 1 4 1.0 1.0 0 0

2 1 2 1.0 1.0 0 0

1 0 5 1.0 1.0 0 0

2 0 4 1.0 1.0 0 0

3 0 0 1.0 1.0 0 0

3 0 1 1.0 1.0 0 0

2 1 3 1


#end_custom_object_intensity_data





#custom_object_texture_factors

_rita_texture_points_number 1_rita_texture_radiations_number 1



data_phase_alpha



loop_

_refln_index_h _refln_index_k _refln_index_l _rita_texture_factor_meas _rita_texture_factor_calc _rita_texture_point_index _rita_texture_radiation_index

1 0 0 1.0 1.0 0 0

0 0 2 1.0 1.0 0 0

1 0 1 1.0 1.0 0 0

1 0 2 1.0 1.0 0 0

1 1 0 1.0 1.0 0 0

1 0 3 1.0 1.0 0 0

2 0 0 1.0 1.0 0 0

1 1 2 1.0 1.0 0 0

2 0 1 1.0 1.0 0 0

0 0 4 1.0 1.0 0 0

2 0 2 1.0 1.0 0 0

1 0 4 1.0 1.0 0 0

2 0 3 1.0 1.0 0 0

2 1 0 1.0 1.0 0 0

2 1 1 1.0 1.0 0 0

1 1 4 1.0 1.0 0 0

2 1 2 1.0 1.0 0 0

1 0 5 1.0 1.0 0 0

2 0 4 1.0 1.0 0 0

3 0 0 1.0 1.0 0 0

3 0 1 1.0 1.0 0 0

2 1 3 1.0 1.0 0 0





data_phase_beta



loop_

_refln_index_h _refln_index_k _refln_index_l _rita_texture_factor_meas _rita_texture_factor_calc _rita_texture_point_index _rita_texture_radiation_index

1 1 0 1.0 1.0 0 0

2 0 0 1.0 1.0 0 0

2 1 1 1.0 1.0 0 0

2 2 0 1.0 1.0 0 0

3 1

## Write an instruction file for the batch mode

To run the analysis in batch mode, an instruction file is required to tell MAUD exactly what analysis to do. This section writes the .ins file.

The number 13 corresponds to the refinement analysis in MAUD (texture mode). The number of iterations sets how many times to run the refinement on each sample.

In [ ]:
# user inputs
number_of_iterations = 7

# name of instruction file
batch_path = pathlib.Path('MAUD_batch_065.ins')

In [5]:
with batch_path.open(mode='w') as output_file:

    output_file.write("loop_\n\
    _riet_analysis_file\n\
    _riet_analysis_iteration_number\n\
    _riet_analysis_wizard_index\n\
    _riet_analysis_fileToSave\n\
    _riet_append_simple_result_to\n\n")
    # note, _riet_meas_datafile_name\n\ removed from above for swapping the files.
    # note, also _riet_append_result_to\n\ but this is only the Rwp value

    for file_number in range (start, end + 1, step):
        output_file.write(f"'/analysis/MAUD_{test_number:03d}_{file_number:05d}.par' "
                          f"{number_of_iterations} 13 '/analysis/MAUD_{test_number:03d}_"
                          f"{file_number:05d}_refined.par' '/analysis/batch_results.txt\'\n")
print(".ins file written.")

.ins file written.


## Running MAUD batch mode

Place the instruction .ins file and a folder for the analysis (containing the newly created .par analysis files) in a sensible location, such as; 


**/Users/mbcx9cd4/Documents/maud/batch/**


_Note, be careful with using a dropbox folder such as /Users/mbcx9cd4/Dropbox\ \(Research\ Group\)/Lightform\ Postdoc/MAUD_


In the terminal navigate to the hidden files within the maud.app using the command; 


**cd /Applications/Maud.app/Contents/Resources/Java**


_Note, you can view these files by right clicking on show package contents._


Then, run the command to start the batch mode;


**java -mx2048M -cp Maud.jar:miscLib.jar:jgaec.jar:ij.jar com.radiographema.MaudText -f /Users/mbcx9cd4/Documents/maud/batch/MAUD_batch_065_mac.ins**


- -mx2048M sets the memory to be used i.e. 512, 1024, 2048, etc.
- Maud.jar:miscLib.jar:jgaec.jar:ij.jar is a class path to the jar files.
- An absolute path to the instruction file must be given.


A pop-up window will then appear, navigate to the .ins instruction file and click open. The batch mode will then run, showing the refinements in the terminal.

## Output many .par analysis files to refine the $\beta$ texture

In [8]:
# number and spacing of files to read
start = 0
end = 10
step = 1

# where the .esg files are written
output_folder = pathlib.Path("analysis")

In [9]:
for image_number in tqdm(range(start, end + 1, step)):

    second_iteration = False
    
    # MAUD analysis file with refined alpha texture
    input_par_path = 'analysis/MAUD_{:03d}_{:05d}_refined.par'.format(test_number,image_number)
   
    # output MAUD analysis file for refining beta texture
    output_par_path = output_folder / pathlib.Path(f'MAUD_{test_number:03d}_{image_number:05d}_refine_beta.par')
    
    with open(input_par_path, 'r') as template_par_file, open(output_par_path, 'w') as new_par_file:
        line = template_par_file.readline() 
            
        while line:
            # remove the max reflection plane indices from the previous refinement
            if '_refln_index_h _refln_index_k _refln_index_l _refln_F_squared_meas _refln_F_squared_calc \
_refln_F_squared_sigma' in line and second_iteration == False:
                new_par_file.write(line.strip('\n'))
                for i in range(0,22):
                    line = template_par_file.readline()
                second_iteration = True
             
            # remove the max reflection plane indices from the previous refinement
            elif '_refln_index_h _refln_index_k _refln_index_l _refln_F_squared_meas _refln_F_squared_calc \
_refln_F_squared_sigma' in line and second_iteration == True:
                new_par_file.write(line.strip('\n'))
                for i in range(0,7):
                    line = template_par_file.readline()
                second_iteration = False
                
            # set refinement of alpha ODF to false
            elif '_rita_odf_refinable true' in line:
                new_par_file.write('_rita_odf_refinable false\n')
                
            elif '#subordinateObject_none tex' in line:
                for l in range (0, 7):
                    line = template_par_file.readline()
                # write metadata for the top of the file
                new_par_file.write('#subordinateObject_E-WIMV\n\n'
                '_pd_proc_ls_pref_orient_corr \'E-WIMV\'\n\n'
                '_rita_generate_symmetry none\n'
                '_rita_wimv_sum_coincidence true\n'
                '_rita_wimv_iteration_max 10\n'
                '_rita_wimv_exponent 0.01\n'
                '_rita_wimv_refl_min_int 0.001\n'
                '_rita_wimv_odf_resolution 10\n'
                '_rita_wimv_tube_projection true\n'
                '_rita_wimv_store_ang_conv true\n'
                '_rita_wimv_odf_coverage_% 0\n'
                '_rita_odf_sharpness ?\n'
                '_rita_wimv_phon_use ?\n'
                '_rita_wimv_tube_weight 0.5\n'
                '_rita_wimv_normalize_pole_figures true\n'
                '_rita_wimv_weigths_exponent 0.5\n'
                '_rita_odf_refinable true\n'
                '_rita_wimv_refl_min_dspacing 0.0\n\n'
                '#custom_object_odf\n'
                'loop_\n'
                '_rita_wimv_odf_values\n')

                for i in range(0, 10):
                    for j in range(0, 10):
                        for k in range (0, 37):
                            new_par_file.write('1.0 ')
                        new_par_file.write('\n')  
                    new_par_file.write('\n')
                new_par_file.write('\n')
                
                new_par_file.write('#end_custom_object_odf\n\n\n'
                '#end_subordinateObject_E-WIMV\n\n\n')
            
            # write the rest of the file
            else:
                new_par_file.write(line)
            
            line = template_par_file.readline()
            
print(f"Written {int((end + 1 - start) / step)} files to '{output_folder}' folder.")


Written 11 files to 'analysis' folder.
